In [35]:
import pandas as pd

gpd = pd.read_csv('data/gdp.csv', index_col='Year')
inflation = pd.read_csv('data/inflation.csv', index_col='date')
unemployment = pd.read_csv('data/unemployment.csv')
yield_data = pd.read_csv('data/yield.csv') #61

### Yield curves

In [36]:
import pandas as pd
import numpy as np

# Convert to datetime
yield_data['Date'] = pd.to_datetime(yield_data['Date'], errors='coerce')

# Replace invalid values with NaN
yield_data.replace(-999.99, pd.NA, inplace=True)

# Drop columns with more than 20% missing values
threshold = 0.2 * len(yield_data)
columns_to_drop = yield_data.columns[yield_data.isnull().sum() > threshold]
yield_data_cleaned = yield_data.drop(columns=columns_to_drop)

# Rename columns 
column_rename_mapping = {
    'BETA0': 'LongTermRate',
    'BETA1': 'Slope',
    'BETA2': 'Curvature1',
    'BETA3': 'Curvature2',
}

for i in range(1, 31):
    column_rename_mapping[f'SVENPY{i:02d}'] = f'SpotRate{i}Y'


for i in range(1, 31):
    column_rename_mapping[f'SVEN1F{i:02d}'] = f'ForwardRate{i}Y'

yield_data_cleaned.rename(columns=column_rename_mapping, inplace=True)

# Drop rows where 'Date' or key features are missing
key_features = ['Date', 'LongTermRate', 'ForwardRate1Y']
yield_data_cleaned = yield_data_cleaned.dropna(subset=key_features)

# Set Date as the index for resampling
yield_data_cleaned.set_index('Date', inplace=True)

# Resample to monthly frequency
monthly_data = yield_data_cleaned.resample('M').mean()

# Interpolate missing values for continuous features
monthly_data_interpolated = monthly_data.interpolate(method='linear', limit_direction='forward', axis=0)

# Logarithmic transformation to reduce skewness for curvature columns
curvature_columns = ['Curvature1', 'Curvature2']
for column in curvature_columns:
    monthly_data_interpolated[column] = monthly_data_interpolated[column].apply(
        lambda x: np.log1p(x) if x > 0 else 0
    )

# Normalize continuous features to the range [0, 1]
continuous_features = [
    col for col in monthly_data_interpolated.columns if col.startswith('LongTermRate') or col.startswith('Slope')
    or col.startswith('Curvature') or col.startswith('SpotRate') or col.startswith('ForwardRate')
]
monthly_data_interpolated[continuous_features] = monthly_data_interpolated[continuous_features].apply(
    lambda x: (x - x.min()) / (x.max() - x.min()), axis=0
)

# Final Output: Display the first few rows of the processed dataset
print(monthly_data_interpolated.head())




            LongTermRate     Slope  Curvature1  Curvature2  ForwardRate1Y  \
Date                                                                        
1961-06-30      0.160110  0.463769    0.000000         0.0       0.233760   
1961-07-31      0.164034  0.446137    0.000000         0.0       0.230211   
1961-08-31      0.169935  0.452569    0.000000         0.0       0.236753   
1961-09-30      0.165211  0.453090    0.000000         0.0       0.237257   
1961-10-31      0.152620  0.455216    0.113939         0.0       0.230314   

            ForwardRate4Y  ForwardRate9Y   SVENF01   SVENF02   SVENF03  ...  \
Date                                                                    ...   
1961-06-30       0.247301            NaN  3.533592  3.934585  3.993169  ...   
1961-07-31       0.254123            NaN  3.436640  3.934995  4.062475  ...   
1961-08-31       0.264753            NaN  3.522839  4.054435  4.201822  ...   
1961-09-30       0.256468            NaN  3.581385  4.014040  4.1

### Unemployment

In [37]:
from sklearn.preprocessing import StandardScaler

print(unemployment.info())  # Check for missing values and data types
print(unemployment.head())  # Preview the data

# Convert to datetime
unemployment['date'] = pd.to_datetime(unemployment['date'], errors='coerce')

unemployment.set_index('date', inplace=True)
unemployment.sort_index(inplace=True)

# Find and Fill all missing data
unemployment.drop(columns=['women_55plus_rate'], inplace=True)
print(unemployment.isnull().sum())
unemployment.interpolate(method='linear', inplace=True) # use linear interpolation for filling NA values

# Normalize data
unemployment_scaler = StandardScaler()
numeric_columns = unemployment.select_dtypes(include=['float64', 'int64']).columns
unemployment[numeric_columns] = unemployment_scaler.fit_transform(unemployment[numeric_columns])



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 917 entries, 0 to 916
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               917 non-null    object 
 1   overall_rate       917 non-null    float64
 2   men_rate           917 non-null    float64
 3   women_rate         917 non-null    float64
 4   men_16_17_rate     917 non-null    float64
 5   women_16_17_rate   917 non-null    float64
 6   men_16_19_rate     917 non-null    float64
 7   women_16_19_rate   917 non-null    float64
 8   men_18_19_rate     917 non-null    float64
 9   women_18_19_rate   917 non-null    float64
 10  men_16_24_rate     917 non-null    float64
 11  women_16_24_rate   917 non-null    float64
 12  men_20_24_rate     917 non-null    float64
 13  women_20_24_rate   917 non-null    float64
 14  men_25plus_rate    917 non-null    float64
 15  women_25plus_rate  917 non-null    float64
 16  men_25_34_rate     917 non

### Inflation

In [38]:
# code